In [3]:
import os  # Operating system interfaces
import tensorflow as tf
from tensorflow import keras    # TensorFlow deep learning framework
from tensorflow.keras import layers, regularizers
# import matplotlib.pyplot as plt                            # Plotting library
# import matplotlib.image as mpimg                           # Image loading and manipulation library
from tensorflow.keras.models import Sequential, Model      # Sequential and Functional API for building models
from tensorflow.keras.optimizers import Adam               # Adam optimizer for model training
from tensorflow.keras.callbacks import EarlyStopping       # Early stopping callback for model training
from tensorflow.keras.regularizers import l1, l2           # L1 and L2 regularization for model regularization
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data augmentation and preprocessing for images
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D, AveragePooling2D, MaxPooling2D, BatchNormalization
# Various types of layers for building neural networks
from tensorflow.keras.applications import DenseNet121, EfficientNetB4, Xception, VGG16, VGG19 # Pre-trained models for transfer learning
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [4]:
# Define the path to the dataset
directory = './Desktop/Disease/train_black_aug_new'

# Load the dataset
train_ds = image_dataset_from_directory(
    directory=directory,
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(224, 224)  # Ensure this matches your model's input size
)


# Define the path to the dataset
directory_val = './Desktop/Disease/val_black_aug_new'

# Load the dataset
val_ds = image_dataset_from_directory(
    directory=directory_val,
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(224, 224)  # Ensure this matches your model's input size
)


Found 22000 files belonging to 10 classes.
Found 4500 files belonging to 10 classes.


In [6]:

conv_base_VGG16 = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(224,224,3),
    pooling='avg'
)

58892288/58889256 [==============================] - 102s 2us/step


In [7]:
conv_base_VGG16.trainable = False

In [8]:
# Add an Input layer to define the input shape for the model
from tensorflow.keras.layers import Input  # Import the Input class

model_VGG = Sequential()
# model_VGG.add(Input(shape=(224, 224, 3)))  # Define the input shape here
model_VGG.add(conv_base_VGG16)
model_VGG.add(Flatten())

model_VGG.add(BatchNormalization())
model_VGG.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model_VGG.add(Dropout(0.35))

model_VGG.add(BatchNormalization())
model_VGG.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model_VGG.add(Dropout(0.35))

model_VGG.add(BatchNormalization())
model_VGG.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model_VGG.add(Dropout(0.35))

model_VGG.add(Dense(10, activation='softmax'))

In [10]:
model_VGG.build(input_shape=(None, 224, 224, 3))

model_VGG.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model_VGG.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               1

In [12]:
# history = model.fit(train_ds,epochs=10,validation_data=validation_ds)
history = model_VGG.fit(train_ds, epochs=70, validation_data=val_ds )

Epoch 1/70
688/688 [==============================] - 297s 408ms/step - loss: 8.2338 - accuracy: 0.4847 - val_loss: 2.1813 - val_accuracy: 0.7211
Epoch 2/70
688/688 [==============================] - 266s 386ms/step - loss: 1.9133 - accuracy: 0.7277 - val_loss: 1.6149 - val_accuracy: 0.7242
Epoch 3/70
688/688 [==============================] - 268s 389ms/step - loss: 1.6301 - accuracy: 0.7264 - val_loss: 1.5850 - val_accuracy: 0.7491
Epoch 4/70
688/688 [==============================] - 279s 404ms/step - loss: 1.6338 - accuracy: 0.7334 - val_loss: 1.6173 - val_accuracy: 0.7409
Epoch 5/70
688/688 [==============================] - 282s 409ms/step - loss: 1.6410 - accuracy: 0.7398 - val_loss: 1.6376 - val_accuracy: 0.7449
Epoch 6/70
688/688 [==============================] - 285s 414ms/step - loss: 1.6490 - accuracy: 0.7339 - val_loss: 1.6432 - val_accuracy: 0.7467
Epoch 7/70
688/688 [==============================] - 288s 418ms/step - loss: 1.6456 - accuracy: 0.7425 - val_loss: 1.6143 -

In [14]:
# Define needed variables
import numpy as np
tr_acc = history.history['accuracy']
tr_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
index_loss = np.argmin(val_loss)
val_lowest = val_loss[index_loss]
index_acc = np.argmax(val_acc)
acc_highest = val_acc[index_acc]

Epochs = [i+1 for i in range(len(tr_acc))]
loss_label = f'best epoch= {str(index_loss + 1)}'
acc_label = f'best epoch= {str(index_acc + 1)}'

# Plot training history
plt.figure(figsize= (20, 8))
plt.style.use('fivethirtyeight')

plt.subplot(1, 2, 1)
plt.plot(Epochs, tr_loss, 'pink', label= 'Training loss')
plt.plot(Epochs, val_loss, 'green', label= 'Validation loss')
plt.scatter(index_loss + 1, val_lowest, s= 150, c= 'blue', label= loss_label)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(Epochs, tr_acc, 'pink', label= 'Training Accuracy')
plt.plot(Epochs, val_acc, 'green', label= 'Validation Accuracy')
plt.scatter(index_acc + 1 , acc_highest, s= 150, c= 'blue', label= acc_label)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout
plt.show()

NameError: name 'plt' is not defined

In [ ]:
# Evaluate the model on the validation data
evaluation_VGG = model_VGG.evaluate(val_ds_VGG)

# Print the evaluation metrics
print("Validation Loss:", evaluation_VGG[0])
print("Validation Accuracy:", evaluation_VGG[1])